In [1]:
import requests
import json
import os
from dotenv import load_dotenv, find_dotenv

# read local .env file
load_dotenv(find_dotenv())

API_KEY = os.environ['ERNIE_API_KEY']
SECRET_KEY = os.environ['ERNIE_SECRET_KEY']

def get_access_token():
    """
    使用 AK，SK 生成鉴权签名（Access Token）
    :return: access_token，或是None(如果错误)
    """
    token_url = "https://aip.baidubce.com/oauth/2.0/token"
    params = {"grant_type": "client_credentials", "client_id": API_KEY, "client_secret": SECRET_KEY}
    return str(requests.post(token_url, params=params).json().get("access_token"))

url = "https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/eb-instant?access_token=" + get_access_token()

In [2]:
headers = {
    'Content-Type': 'application/json'
}

In [3]:
payload = json.dumps({
    "messages": [
    ]
})
payload_dict = json.loads(payload)

In [4]:
def ask(content):
    new_query = {
        "role": "user",
        "content": content
    }
    payload_dict["messages"].append(new_query)
    params = json.dumps(payload_dict)

    response = requests.request("POST", url, headers=headers, data=params)
    result = json.loads(response.text)['result']

    new_history_message = {
        "role": "assistant",
        "content": result
    }
    payload_dict["messages"].append(new_history_message)
    return result

# prompt 没调试好的时候，返回数据会被 markdown 语法渲染，所以需要提取 json
def extract_json_from_markdown(md_string):
    json_string = md_string.split('```json')[1].split('```')[0].strip()
    return json.loads(json_string)

In [5]:
question = """
、、、
人要是行，干一行行一行，一行行行行行；

要是不行，干一行不行一行,一行不行行行不行
、、、
请对上面的语句充分理解语义然后进行分词。
例如，“干一行行一行”，应该分词为“干一行(hang)”、“行(xing)”、“一行(hang)”。
注意，对"行"字加上音标，比如 "一行(hang)"，"行行(hanghang)"。
"""
answer = ask(question)
print(answer)

人要 是 行 ， 干 一 行 行 一 行 ， 一 行 行 行 行 行 ； 

要 是不 行 ， 干 一 行 不行 一 行 ， 一 行 不行 行 行 不 行


### 分词效果一般

In [6]:
question = """
请提供一个自然语言信息提取的过程。您的回答应该包括以下内容：介绍自然语言信息提取的定义和用途，描述信息提取的步骤，如分词和语法分析，以及解释如何训练模型以实现信息提取。请提供具体的例子和清晰的说明，以便读者能够理解这个过程。

请注意，您的回答应该具有一定的灵活性，以允许各种相关但有创意的步骤和技术。
"""
answer = ask(question)
print(answer)

自然语言信息提取（NLP）是指从自然语言文本中自动提取有用的信息。NLP在许多领域都有广泛的应用，如自然语言处理、机器翻译、信息检索、问答系统等。

自然语言信息提取通常包括以下几个步骤：

1. 分词：将自然语言文本分成一个个独立的词语或词组。分词是NLP中最基础也是最关键的步骤之一，因为正确的分词可以极大地影响后续的步骤，如词性标注、句法分析等。
2. 词性标注：给定一个词语，标注出其应该使用的词性。例如，“book”应该标注为名词，“like”应该标注为动词。词性标注可以帮助我们更好地理解句子的结构和意义。
3. 句法分析：给定一个句子，分析其句法结构，即各个词语之间的语法关系。句法分析可以帮助我们更好地理解句子的意思，并识别出句中的重要信息，如主语、谓语、宾语等。
4. 实体识别：识别文本中出现的实体，如人名、地名、组织机构等。实体识别可以帮助我们更好地理解文本中的信息，并在一些应用中，如问答系统和日历应用，可以用来执行一些操作，如拨打电话或发送电子邮件。
5. 情感分析：对文本中的情感进行分析，如积极、消极或中立等。情感分析可以帮助我们了解人们对某个话题或事件的看法和态度，从而更好地理解和预测人类行为。

训练模型以实现信息提取通常有两种方法：有监督学习和无监督学习。有监督学习需要大量的标注数据，通过训练模型来识别出文本中的各种结构、实体和情感等信息。而无监督学习则不需要大量的标注数据，可以通过聚类、分类等算法来识别出文本中的信息。

以电影评论数据集为例，我们可以进行以下信息提取：

1. 电影名称：从文本中识别出电影名称。
2. 电影演员：识别出电影中的演员，包括演员的姓名和角色。
3. 电影导演：识别出电影的导演，包括导演的姓名。
4. 电影类型：识别出电影的类型，如动作片、喜剧片、爱情片等。
5. 电影评分：识别出电影的评分，通常使用数字表示。
6. 电影简介：从文本中提取出电影的故事简介。

这些信息都可以通过使用自然语言信息提取技术来自动提取出来。


### 上面的 prompt 是经过 [promptperfect](https://promptperfect.jina.ai/home) 美化的，可以看到结果很不错

In [7]:
user_content = "给刘德华（电话13400000000）送一斤吧。还有个叫张学友，电话13500000000也送。好了，下单吧。"
question = f"""
请根据输入信息，通过命名实体识别（NER）抽取特定结构的数据。
抽取的数据结构如下：
[{{"receiverName": "", "receiverMobile": ""}}]

输入信息：
'''
{user_content}
'''
抽取的数据为：
"""
answer = ask(question)
print(answer)

[{"receiverName": "刘德华", "receiverMobile": "13400000000"}, {"receiverName": "张学友", "receiverMobile": "13500000000"}]


### 经过调整 prompt 后，正确抽取了参数。这种情况加上 few-shot 会更好。

In [8]:
question = "我最开始问你的问题是什么？"
answer = ask(question)
print(answer)

您最初问的问题是："、、、人要是行，干一行行一行，一行行行行行；要是不行，干一行不行一行,一行不行行行不行"


### 自动精简了第一个问题的内容，但是不够完整

In [9]:
from pprint import pprint
pprint(payload_dict)

{'messages': [{'content': '\n'
                          '、、、\n'
                          '人要是行，干一行行一行，一行行行行行；\n'
                          '\n'
                          '要是不行，干一行不行一行,一行不行行行不行\n'
                          '、、、\n'
                          '请对上面的语句充分理解语义然后进行分词。\n'
                          '例如，“干一行行一行”，应该分词为“干一行(hang)”、“行(xing)”、“一行(hang)”。\n'
                          '注意，对"行"字加上音标，比如 "一行(hang)"，"行行(hanghang)"。\n',
               'role': 'user'},
              {'content': '人要 是 行 ， 干 一 行 行 一 行 ， 一 行 行 行 行 行 ； \n'
                          '\n'
                          '要 是不 行 ， 干 一 行 不行 一 行 ， 一 行 不行 行 行 不 行',
               'role': 'assistant'},
              {'content': '\n'
                          '请提供一个自然语言信息提取的过程。您的回答应该包括以下内容：介绍自然语言信息提取的定义和用途，描述信息提取的步骤，如分词和语法分析，以及解释如何训练模型以实现信息提取。请提供具体的例子和清晰的说明，以便读者能够理解这个过程。\n'
                          '\n'
                          '请注意，您的回答应该具有一定的灵活性，以允许各种相关但有创意的步骤和技术。\n',
               'role': 'user'},
              {'cont